In [32]:
from pathlib import Path
import mre
import numpy as np
from tomato.audio.pitchdistribution import PitchDistribution
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

In [48]:
annotations = mre.data.Annotation.from_mlflow()
annotations.head()

y = np.array(annotations.data.makam)

X = np.zeros((len(pcd_files), 160))
mbids = []
for ii, pf in enumerate(pcd_files):
    X[ii, :] = PitchDistribution.from_json(pcd_files[ii]).vals
    mbids.append(Path(pcd_files[ii]).stem)
    
y = annotations.data.set_index("mbid").loc[mbids].makam.values

In [51]:
n_splits = 10
random_state = 1916  # death of Tanburi Cemil Bey for reproducibility

inner = StratifiedKFold(n_splits=10, random_state=random_state, shuffle=True)
svm = SVC()
knn = KNeighborsClassifier()

for ii, fold in enumerate(inner.split(X=X, y=y)):
    svm.fit(X[fold[0], :], y[fold[0]])
    
    print(svm.score(X[fold[1], :], y[fold[1]]))

0.66
0.63
0.71
0.65
0.79
0.7
0.76
0.65
0.7
0.72


In [ ]:
n_splits = 10
n_repeats = 10
random_state = 1916  # death of Tanburi Cemil Bey for reproducibility

outer = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
inner = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)

In [ ]:
repeats = []
for oo in outer.split(X=ids, y=y):
    test_id = ids[oo[1]]
    test_y = y[oo[1]]
    
    inner_id = ids[oo[0]]
    inner_y = y[oo[0]]
    
    cv_folds = [
        {
            "train": {
                "ids": ids[fold[0]],
                "y": y[fold[0]]
            },
            "validation": {
                "ids": ids[fold[1]],
                "y": y[fold[1]]
            }
        }
        for fold in inner.split(X=inner_id, y=inner_y)
    ]
    repeats.append({
        "cross_validation": cv_folds, 
        "test": {
            "ids": test_id,
            "y": test_y
        }
    })

In [ ]:
np.unique(repeats[0]["cross_validation"][0]["validation"]["y"], return_counts=True)

In [ ]:
clf.best_estimator_

In [ ]:
from sklearn.datasets import load_iris
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np

print(__doc__)

# Number of random trials
NUM_TRIALS = 30

# Load the dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

# Set up possible values of parameters to optimize over
p_grid = {"C": [1, 10, 100],
          "gamma": [.01, .1]}

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC(kernel="rbf")

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
    inner_cv = KFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv)
    
    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X_iris, y=y_iris, cv=outer_cv)
    nested_scores[i] = nested_score.mean()
    
    clf.fit(X_iris, y_iris)
    non_nested_scores[i] = clf.best_score_

score_difference = non_nested_scores - nested_scores

print("Average difference of {:6f} with std. dev. of {:6f}."
      .format(score_difference.mean(), score_difference.std()))

# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on Iris Dataset",
          x=.5, y=1.1, fontsize="15")

# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

In [ ]:
clf.cv_results_